# Ibrahim's Custom Feature Engineering

In [ ]:
!pip install -q antropy statsmodels librosa

In [ ]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
from scipy.integrate import simpson
import antropy as ant
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
import pywt

class EEGFeatureExtractor:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    # --- Helper: 1D Local Binary Pattern ---
    def _lbp_1d(self, x):
        code = np.zeros(len(x)-2)
        for i in range(1, len(x)-1):
            b1 = 1 if x[i-1] >= x[i] else 0
            b2 = 1 if x[i+1] >= x[i] else 0
            code[i-1] = b1 * 2 + b2 * 1 
        return np.mean(code) 
    
    # --- Helper: EMG Envelope ---
    def _emg_envelope(self, x):
        sos = signal.butter(4, 30, 'hp', fs=self.sfreq, output='sos')
        filt = signal.sosfilt(sos, x)
        rect = np.abs(filt)
        sos_lp = signal.butter(4, 10, 'lp', fs=self.sfreq, output='sos')
        envelope = signal.sosfilt(sos_lp, rect)
        return np.mean(envelope)
    
        # --- Helper: safe Weighted Permutation Entropy ---
    def _weighted_perm_entropy(self, x, order=3, delay=1, normalize=True):
        """
        Wrapper around ant.weighted_permutation_entropy with simple safety checks.
        """
        x = np.asarray(x)
        # Need enough points for embedding
        if len(x) < order * delay + 1:
            return np.nan
        # Constant signal -> zero entropy
        if np.allclose(x, x[0]):
            return 0.0
        try:
            return ant.weighted_permutation_entropy(
                x, order=order, delay=delay, normalize=normalize
            )
        except Exception:
            return np.nan

    # --- Helper: safe Fuzzy Entropy ---
    def _fuzzy_entropy(self, x, m=2, r=None):
        """
        Wrapper around ant.fuzzy_entropy with automatic r and safety checks.
        """
        x = np.asarray(x)
        if len(x) < m + 2:
            return np.nan
        if r is None:
            r = 0.2 * np.std(x)  # common heuristic
        # If std ~ 0, r will be tiny and can blow up numerics
        if r == 0 or np.isnan(r):
            return 0.0
        try:
            return ant.fuzzy_entropy(x, m=m, r=r)
        except Exception:
            return np.nan

    # --- Helper: safe Distribution Entropy ---
    def _distribution_entropy(self, x, m=2, tau=1, normalize=True):
        """
        Wrapper around ant.distribution_entropy with safety checks.
        """
        x = np.asarray(x)
        if len(x) < m * tau + 1:
            return np.nan
        # Constant signals -> low/zero entropy
        if np.allclose(x, x[0]):
            return 0.0
        try:
            return ant.distribution_entropy(
                x, m=m, tau=tau, normalize=normalize
            )
        except Exception:
            return np.nan
        
        # --- Helper: aEEG envelope (core processing) ---
    def _aeeg_envelope(self, x, sf=None):
        """
        Compute the aEEG envelope:
            1. 2-15 Hz band-pass filter
            2. Rectify
            3. Low-pass filter (~0.5 Hz)

        Returns:
            envelope : np.ndarray or None if too short / invalid
        """
        x = np.asarray(x)
        sf = self.sfreq if sf is None else sf

        # Require at least 2 seconds of data
        if len(x) < 2 * sf:
            return None

        # 1. Band-pass 2–15 Hz
        sos = signal.butter(4, [2, 15], btype='bandpass', fs=sf, output='sos')
        bp = signal.sosfilt(sos, x)

        # 2. Rectify
        rect = np.abs(bp)

        # 3. Smooth with low-pass (0.5 Hz ≈ 2 s time constant)
        cutoff = 0.5
        sos_lp = signal.butter(4, cutoff, btype='low', fs=sf, output='sos')
        envelope = signal.sosfilt(sos_lp, rect)

        return envelope

    # --- Helper: aEEG feature block (clinical-style) ---
    def _aeeg_features(self, x, sf=None):
        """
        Compute a set of aEEG-derived features similar to clinical aEEG:
            - aeeg_mean: mean envelope amplitude
            - aeeg_lower_margin: 5th percentile of envelope
            - aeeg_upper_margin: 95th percentile of envelope
            - aeeg_bandwidth: upper - lower
            - aeeg_log_mean: mean log10 amplitude
            - aeeg_log_bandwidth: log10(upper) - log10(lower)

        Returns:
            dict of feature_name -> value (np.nan if cannot compute)
        """
        sf = self.sfreq if sf is None else sf
        env = self._aeeg_envelope(x, sf=sf)

        feats = {
            'aEEG_mean': np.nan,
            'aEEG_lower_margin': np.nan,
            'aEEG_upper_margin': np.nan,
            'aEEG_bandwidth': np.nan,
            'aEEG_log_mean': np.nan,
            'aEEG_log_bandwidth': np.nan,
        }

        if env is None:
            return feats

        # Guard against non-positive values in log
        eps = 1e-6
        env_safe = env + eps

        lower = np.percentile(env, 5)
        upper = np.percentile(env, 95)
        mean_amp = float(np.mean(env))

        feats['aEEG_mean'] = mean_amp
        feats['aEEG_lower_margin'] = float(lower)
        feats['aEEG_upper_margin'] = float(upper)
        feats['aEEG_bandwidth'] = float(upper - lower)

        feats['aEEG_log_mean'] = float(np.mean(np.log10(env_safe)))
        feats['aEEG_log_bandwidth'] = float(
            np.log10(upper + eps) - np.log10(lower + eps)
        )

        return feats


    # --- Helper: Hurst exponent (rescaled range / R/S analysis) ---
    def _hurst_exponent(self, x):
        """
        Computes the Hurst exponent using classical R/S analysis.

        Returns:
            H (float): Hurst exponent in [0, 1] or np.nan on error.
        """
        x = np.asarray(x, dtype=float)

        # Need sufficient data
        if len(x) < 200:
            return np.nan
        
        # Constant signals → H = 0.5 (pure noise-like)
        if np.allclose(x, x[0]):
            return 0.5
        
        # Segment sizes (log-spaced)
        N = len(x)
        segment_sizes = np.floor(np.logspace(1.3, np.log10(N / 2), num=10)).astype(int)
        segment_sizes = np.unique(segment_sizes)
        
        RS_vals = []
        sizes_used = []

        for size in segment_sizes:
            if size < 10 or size >= N:
                continue

            n_segments = N // size
            if n_segments < 2:
                continue

            RS_per_seg = []

            for i in range(n_segments):
                seg = x[i*size:(i+1)*size]
                mean_seg = np.mean(seg)
                Y = np.cumsum(seg - mean_seg)
                R = np.max(Y) - np.min(Y)
                S = np.std(seg)

                if S == 0:
                    continue

                RS_per_seg.append(R / S)

            if len(RS_per_seg) > 0:
                RS_vals.append(np.mean(RS_per_seg))
                sizes_used.append(size)

        # Need at least 2 points for regression
        if len(RS_vals) < 2:
            return np.nan

        # Log–log linear regression
        log_sizes = np.log10(sizes_used)
        log_RS = np.log10(RS_vals)

        slope, _ = np.polyfit(log_sizes, log_RS, 1)
        H = float(slope)

        return H



    def extract_time_domain(self, x):
        f = {}
        # 1. Statistics
        f['mean'] = np.mean(x)
        f['std'] = np.std(x)
        f['median'] = np.median(x)
        f['max'] = np.max(x)
        f['min'] = np.min(x)
        f['range'] = f['max'] - f['min']
        f['iqr'] = stats.iqr(x)
        f['kurtosis'] = stats.kurtosis(x)
        f['skew'] = stats.skew(x)
        f['Q1'] = np.percentile(x, 25)
        f['Q3'] = np.percentile(x, 75)
        
        # 2. Differences
        diff1 = np.diff(x)
        diff2 = np.diff(diff1)
        f['mean_abs_diff1_raw'] = np.mean(np.abs(diff1))
        f['mean_abs_diff2_raw'] = np.mean(np.abs(diff2))
        f['mean_abs_diff1_norm'] = f['mean_abs_diff1_raw'] / (f['std'] + 1e-6)
        f['mean_abs_diff2_norm'] = f['mean_abs_diff2_raw'] / (f['std'] + 1e-6)
        
        # 3. AR Coefficients (Order 4)
        try:
            res = AutoReg(x, lags=4, trend='n').fit()
            for i, c in enumerate(res.params):
                f[f'ar_coeff_{i}'] = c
        except:
            for i in range(4): f[f'ar_coeff_{i}'] = 0
            
        # 4. Nonlinear / Entropy 
        f['higuchi_fd'] = ant.higuchi_fd(x)
        f['svd_entropy'] = ant.svd_entropy(x, normalize=True)
        f['perm_entropy'] = ant.perm_entropy(x, normalize=True)
        f['app_entropy'] = ant.app_entropy(x)
        f['sample_entropy'] = ant.sample_entropy(x)
        f['petrosian_fd'] = ant.petrosian_fd(x)
        f['katz_fd'] = ant.katz_fd(x)
        f['lziv_complexity'] = ant.lziv_complexity(x > np.mean(x), normalize=True)
        f['DFA'] = ant.detrended_fluctuation(x)
        f['shannon_entropy'] = stats.entropy(np.abs(x) + 1e-6)
        f['weighted_permutation_entropy'] = self._weighted_perm_entropy(x, normalize=True)
        f['fuzzy_entropy'] = self._fuzzy_entropy(x)
        f['distribution_entropy'] = self._distribution_entropy(x, normalize=True)
        
        # 5. Hjorth
        f['hjorth_mob'], f['hjorth_comp'] = ant.hjorth_params(x)
        f['hjorth_act'] = f['std']**2
        
        # 6. Energy/Power
        f['rms'] = np.sqrt(np.mean(x**2))
        f['line_length'] = np.sum(np.abs(diff1))
        f['nonlinear_energy'] = np.mean(x[1:-1]**2 - x[:-2] * x[2:])
        
        # 7. Patterns
        f['lbp_mean'] = self._lbp_1d(x)
        f['zero_crossings'] = ant.num_zerocross(x)
        f['emg_envelope_mean'] = self._emg_envelope(x)
        aeeg_feats = self._aeeg_features(x)
        f.update(aeeg_feats)
        
        return f

    def extract_frequency_domain(self, x):
        f = {}
        freqs, psd = signal.welch(x, self.sfreq, nperseg=min(len(x), 256))
        psd_norm = psd / (np.sum(psd) + 1e-6)
        
        f['peak_freq'] = freqs[np.argmax(psd)]
        f['spectral_entropy'] = ant.spectral_entropy(x, self.sfreq, method='welch', normalize=True)
        f['weighted_mean_freq'] = np.sum(freqs * psd_norm)
        f['median_freq'] = freqs[np.where(np.cumsum(psd_norm) >= 0.5)[0][0]]
        f['bandwidth'] = np.sqrt(np.sum(((freqs - f['weighted_mean_freq'])**2) * psd_norm))
        f['spectral_edge_freq_95'] = freqs[np.where(np.cumsum(psd_norm) >= 0.95)[0][0]]
        f['spectral_edge_freq_90'] = freqs[np.where(np.cumsum(psd_norm) >= 0.90)[0][0]]
        f['intensity_weighted_mean_freq'] = np.sum(freqs * (psd**2)) / (np.sum(psd**2) + 1e-6)
        
        bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 80)}
        total_power = 0
        for band, (low, high) in bands.items():
            idx = np.logical_and(freqs >= low, freqs <= high)
            power = simpson(y = psd[idx], x = freqs[idx])
            f[f'power_{band}'] = power
            total_power += power
            
        f['total_power'] = total_power
        f['ratio_delta_alpha'] = f['power_delta'] / (f['power_alpha'] + 1e-6)
        f['ratio_theta_beta'] = f['power_theta'] / (f['power_beta'] + 1e-6)
        f['hurst_exponent'] = self._hurst_exponent(x)
        
        f['rel_power_delta'] = f['power_delta'] / (total_power + 1e-6)
        f['rel_power_theta'] = f['power_theta'] / (total_power + 1e-6)
        f['rel_power_alpha'] = f['power_alpha'] / (total_power + 1e-6)
        f['rel_power_beta'] = f['power_beta'] / (total_power +  1e-6)
        f['rel_power_gamma'] = f['power_gamma'] / (total_power + 1e-6)
        
        f['ratio_theta_alpha'] = f['power_theta'] / (f['power_alpha'] + 1e-6)
        f['ratio_alpha_beta'] = f['power_alpha'] / (f['power_beta'] + 1e-6)
        f['ratio_beta_gamma'] = f['power_beta'] / (f['power_gamma'] + 1e-6)
        
        f['ratio_slow_fast'] = (f['power_delta'] + f['power_theta']) / (f['power_alpha'] + f['power_beta'] + f['power_gamma'] + 1e-6)
        f['ratio_slow_alpha_beta'] = (f['power_delta'] + f['power_theta']) / (f['power_alpha'] + f['power_beta'] + 1e-6)
        f['ratio_alpha_total'] = f['power_alpha'] / (total_power + 1e-6)
        
        # Hilbert Features
        analytic = signal.hilbert(x)
        amplitude = np.abs(analytic)
        phase = np.unwrap(np.angle(analytic))
        inst_freq = (np.diff(phase) / (2.0*np.pi) * self.sfreq)
        f['hilbert_amp_mean'] = np.mean(amplitude)
        f['hilbert_freq_mean'] = np.mean(inst_freq)
        
        return f

    def extract_time_frequency_domain(self, x):
        f = {}
        # Short-Time Fourier Transform (STFT) / Spectrogram
        f_stft, t_stft, Zxx = signal.stft(x, fs=self.sfreq, nperseg=min(len(x)//4, 64))
        Sxx = np.abs(Zxx) # Magnitude spectrogram

        # 1. Spectrogram Statistics
        f['spectrogram_mean'] = np.mean(Sxx)
        f['spectrogram_std'] = np.std(Sxx)
        f['log_variance'] = np.var(np.log(Sxx + 1e-6))
        
        # 2. Frequency tracking over time
        # Mean frequency at each time step
        mean_freq_over_time = np.sum(Sxx * f_stft[:, None], axis=0) / (np.sum(Sxx, axis=0) + 1e-6)
        f['mean_frequency_tf'] = np.mean(mean_freq_over_time)
        f['median_frequency_tf'] = np.median(mean_freq_over_time)
        
        # 3. Peak Frequency stability
        # Index of max power at each time step
        peak_freq_indices = np.argmax(Sxx, axis=0)
        peak_freqs = f_stft[peak_freq_indices]
        f['peak_frequency_timefreq_mean'] = np.mean(peak_freqs)
        f['peak_frequency_timefreq_std'] = np.std(peak_freqs)

        return f

    def extract_decomposition_domain(self, x):
        f = {}
        
        # 1. Discrete Wavelet Transform (DWT)
        # Level 4 decomposition with Daubechies 4
        coeffs = pywt.wavedec(x, 'db4', level=4)
        # coeffs: [cA4, cD4, cD3, cD2, cD1]
        coeff_names = ['A4', 'D4', 'D3', 'D2', 'D1']
        
        for name, c in zip(coeff_names, coeffs):
            f[f'dwt_{name}_energy'] = np.sum(c**2)
            f[f'dwt_{name}_mean'] = np.mean(c)
            f[f'dwt_{name}_std'] = np.std(c)
            # Approximate entropy of coefficients
            f[f'dwt_{name}_entropy'] = stats.entropy(np.abs(c) + 1e-6)

        # 2. Continuous Wavelet Transform (CWT) Proxy
        # Using Ricker wavelet (Mexican Hat) for scales 1-30
        widths = np.arange(1, 31)
        cwtmatr, _ = pywt.cwt(x, widths, 'mexh')
        f['cwt_max_power'] = np.max(cwtmatr**2)
        f['cwt_mean_power'] = np.mean(cwtmatr**2)

        # 3. ARMA / ARIMA Model
        # Using a simple ARMA(1,1) to capture linear dynamics
        # Note: Full ARIMA fitting is slow; we use conditional sum of squares or smaller iter
        try:
            # We fit a simple AR(1) here as proxy for ARMA to save time on 140k samples
            # or we can try simple 1-step forecast error
            model = AutoReg(x, lags=1, trend='c').fit()
            f['arma_ar1_coeff'] = model.params[1]
            f['arma_resid_std'] = np.std(model.resid)
        except:
            f['arma_ar1_coeff'] = 0
            f['arma_resid_std'] = 0
            
        return f
    
    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_time_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))
        
        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())


print("EEGFeatureExtractor updated with Time, Frequency, Time-Freq, and Decomposition domains.")

extractor = EEGFeatureExtractor()
dummy = np.random.randn(1000)  # 4 seconds at 250Hz

_ = extractor.list_feature_names(dummy)

EEGFeatureExtractor updated with Time, Frequency, Time-Freq, and Decomposition domains.
Total feature count: 108
Feature names:
mean
std
median
max
min
range
iqr
kurtosis
skew
Q1
Q3
mean_abs_diff1_raw
mean_abs_diff2_raw
mean_abs_diff1_norm
mean_abs_diff2_norm
ar_coeff_0
ar_coeff_1
ar_coeff_2
ar_coeff_3
higuchi_fd
svd_entropy
perm_entropy
app_entropy
sample_entropy
petrosian_fd
katz_fd
lziv_complexity
DFA
spectral_entropy
shannon_entropy
weighted_permutation_entropy
fuzzy_entropy
distribution_entropy
hjorth_mob
hjorth_comp
hjorth_act
rms
line_length
nonlinear_energy
lbp_mean
zero_crossings
emg_envelope_mean
aEEG_mean
aEEG_lower_margin
aEEG_upper_margin
aEEG_bandwidth
aEEG_log_mean
aEEG_log_bandwidth
peak_freq
weighted_mean_freq
median_freq
bandwidth
spectral_edge_freq_95
spectral_edge_freq_90
intensity_weighted_mean_freq
power_delta
power_theta
power_alpha
power_beta
power_gamma
total_power
ratio_delta_alpha
ratio_theta_beta
hurst_exponent
rel_power_delta
rel_power_theta
rel_power_alpha

In [21]:
import numpy as np
import scipy.signal as signal
import scipy.fftpack as fftpack
import scipy.stats as stats
import librosa
import pywt
from scipy.integrate import simpson

class ECGFeatureExtractor:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    # --- Helper: Pan-Tompkins QRS Detector (Simplified) ---
    def _detect_qrs(self, x):
        # A. Bandpass Filter (5-15Hz)
        sos = signal.butter(3, [5, 15], 'bandpass', fs=self.sfreq, output='sos')
        filtered = signal.sosfilt(sos, x)
        
        # B. Derivative
        diff = np.diff(filtered)
        
        # C. Squaring
        squared = diff ** 2
        
        # D. Moving Window Integration (150ms)
        window_size = int(0.15 * self.sfreq)
        integrated = np.convolve(squared, np.ones(window_size)/window_size, mode='same')
        
        # E. Peak Finding
        # Height threshold relative to max signal
        thresh = np.mean(integrated) * 2
        peaks, _ = signal.find_peaks(integrated, height=thresh, distance=int(0.2*self.sfreq))
        
        return peaks
    
    # --- Helper: Normalized energy in peak band & harmonics ---
    def _normalized_peak_harmonic_energy(self, freqs, psd, n_harmonics=3, band_width=0.5):
        """
        Compute normalized energy around the dominant frequency and its harmonics.

        Parameters
        ----------
        freqs : np.ndarray
            Frequency vector from PSD (e.g., from signal.welch).
        psd : np.ndarray
            Power spectral density values corresponding to freqs.
        n_harmonics : int
            Number of harmonics to include (fundamental + this many-1).
        band_width : float
            Half-width (in Hz) of each band around k*f0, i.e. [k*f0 - band_width, k*f0 + band_width].

        Returns
        -------
        norm_peak_energy : float
            Normalized energy in the fundamental band.
        norm_harmonics_energy : float
            Normalized energy in all harmonic bands (2*f0, 3*f0, ...).
        norm_total_peak_harmonics : float
            Normalized energy in fundamental + harmonics together.
        """
        freqs = np.asarray(freqs)
        psd = np.asarray(psd)

        # Total spectral energy
        total_power = simpson(y = psd, x = freqs)
        if total_power <= 0 or len(freqs) == 0:
            return 0.0, 0.0, 0.0

        # Fundamental frequency (dominant peak)
        peak_idx = np.argmax(psd)
        f0 = freqs[peak_idx]

        energy_fund = 0.0
        energy_harm = 0.0

        for k in range(1, n_harmonics + 1):
            fk = k * f0
            # If the band is entirely beyond Nyquist, stop
            if fk - band_width > freqs[-1]:
                break

            band_mask = (freqs >= fk - band_width) & (freqs <= fk + band_width)
            if not np.any(band_mask):
                continue

            band_energy = simpson(y = psd[band_mask], x = freqs[band_mask])

            if k == 1:
                energy_fund += band_energy
            else:
                energy_harm += band_energy

        eps = 1e-12
        norm_peak = energy_fund / (total_power + eps)
        norm_harm = energy_harm / (total_power + eps)
        norm_total = (energy_fund + energy_harm) / (total_power + eps)

        return float(norm_peak), float(norm_harm), float(norm_total)


    def extract_time_domain(self, x):
        f = {}
        # Basic Stats
        f['mean'] = np.mean(x)
        f['std'] = np.std(x)
        f['median'] = np.median(x)
        f['range'] = np.max(x) - np.min(x)
        f['skewness'] = stats.skew(x)
        f['kurtosis'] = stats.kurtosis(x)
        f['rms'] = np.sqrt(np.mean(x**2))
        f['max'] = np.max(x)
        f['min'] = np.min(x)
        f['IQR'] = stats.iqr(x)
        f['Q1'] = np.percentile(x, 25)
        f['Q3'] = np.percentile(x, 75)
        
        # QRS & HR Features
        r_peaks = self._detect_qrs(x)
        
        if len(r_peaks) > 1:
            rr_intervals = np.diff(r_peaks) / self.sfreq # in seconds
            f['heart_rate'] = 60.0 / np.mean(rr_intervals) # BPM
            f['heart_rate_variability'] = np.std(rr_intervals) # SDNN
            f['rr_range'] = np.max(rr_intervals) - np.min(rr_intervals)
            f['qrs_count'] = len(r_peaks)
        else:
            f['heart_rate'] = 0
            f['heart_rate_variability'] = 0
            f['rr_range'] = 0
            f['qrs_count'] = len(r_peaks)

        # Linear Predictive Coding (LPC)
        # Order 4 as requested
        try:
            # Librosa lpc expects 1D array
            a_lpc = librosa.lpc(x, order=4)
            for i, c in enumerate(a_lpc[1:]): # Skip a[0] which is 1
                f[f'lpc_coeff_{i}'] = c
            lpc_resid = signal.lfilter(a_lpc, 1, x)
            f['lpc_resid_std'] = np.std(lpc_resid)
            f['lpc_resid_var'] = np.var(lpc_resid)
            f['lpc_resid_energy'] = np.sum(lpc_resid**2)
        except:
            for i in range(4): f[f'lpc_coeff_{i}'] = 0
            
        return f

    def extract_frequency_domain(self, x):
        f = {}
        # FFT / PSD
        freqs, psd = signal.welch(x, self.sfreq, nperseg=min(len(x), 256))
        f['dominant_freq'] = freqs[np.argmax(psd)]
        f['total_power'] = np.sum(psd)
        
        norm_peak, norm_harm, norm_total = self._normalized_peak_harmonic_energy(
        freqs, psd, n_harmonics=3, band_width=0.5)
        f['norm_peak_band_energy'] = norm_peak
        f['norm_harmonics_energy'] = norm_harm
        f['norm_peak_plus_harmonics_energy'] = norm_total
        
        # MFCC (Mel-frequency cepstral coefficients)
        # Treating ECG as "audio" for texture analysis is common
        try:
            sig_len = len(x)
            # Choose an n_fft that is not larger than the signal and is reasonably sized
            n_fft = min(1024, sig_len)
            hop_length = max(1, n_fft // 4)

            mfcc = librosa.feature.mfcc(
                y=x.astype(float),
                sr=self.sfreq,
                n_mfcc=5,
                n_fft=n_fft,
                hop_length=hop_length,
            )
            mfcc_mean = np.mean(mfcc, axis=1)
            for i, c in enumerate(mfcc_mean):
                f[f'mfcc_{i}'] = c
        except Exception:
            for i in range(5):
                f[f'mfcc_{i}'] = 0

            
        # Discrete Cosine Transform (DCT)
        dct_val = fftpack.dct(x, type=2, norm='ortho')
        f['dct_mean'] = np.mean(np.abs(dct_val))
        f['dct_var'] = np.var(dct_val)
        
        # Hilbert Features
        analytic = signal.hilbert(x)
        amplitude = np.abs(analytic)
        phase = np.unwrap(np.angle(analytic))
        inst_freq = (np.diff(phase) / (2.0*np.pi) * self.sfreq)
        f['hilbert_amp_mean'] = np.mean(amplitude)
        f['hilbert_freq_mean'] = np.mean(inst_freq)
        
        return f

    def extract_time_frequency_domain(self, x):
        f = {}
        # STFT / Spectrogram features
        # (Used as proxy for heavy WVD/CWD computations)
        f_stft, t_stft, Zxx = signal.stft(x, fs=self.sfreq, nperseg=min(len(x)//4, 64))
        Sxx = np.abs(Zxx)
        
        f['stft_energy_mean'] = np.mean(Sxx)
        f['stft_energy_std'] = np.std(Sxx)
        
        # Peak frequency tracking
        peak_freqs = f_stft[np.argmax(Sxx, axis=0)]
        f['stft_peak_freq_mean'] = np.mean(peak_freqs)
        
        return f

    def extract_decomposition_domain(self, x):
        f = {}
        # Wavelet Transform (DWT)
        coeffs = pywt.wavedec(x, 'db4', level=4)
        coeff_names = ['A4', 'D4', 'D3', 'D2', 'D1']
        for name, c in zip(coeff_names, coeffs):
            f[f'dwt_{name}_energy'] = np.sum(c**2)
            f[f'dwt_{name}_std'] = np.std(c)
            
        # SVD (Singular Value Decomposition)
        # SVD on a 1D signal is just scalar, usually done on trajectory matrix (Hankel)
        # We'll do SVD on the spectrogram matrix Sxx for robustness
        try:
            _, _, Zxx = signal.stft(x, fs=self.sfreq, nperseg=64)
            Sxx = np.abs(Zxx)
            U, s, Vh = np.linalg.svd(Sxx, full_matrices=False)
            # Top 3 Singular Values
            for i in range(min(3, len(s))):
                f[f'svd_val_{i}'] = s[i]
        except:
            for i in range(3): f[f'svd_val_{i}'] = 0
            
        return f
    
    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_time_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))
        
        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())

print("ECGFeatureExtractor class defined.")

extractor = ECGFeatureExtractor()
dummy = np.random.randn(1000)  # 4 seconds at 250Hz

_ = extractor.list_feature_names(dummy)

ECGFeatureExtractor class defined.
Total feature count: 53
Feature names:
mean
std
median
range
skewness
kurtosis
rms
max
min
IQR
Q1
Q3
heart_rate
heart_rate_variability
rr_range
qrs_count
lpc_coeff_0
lpc_coeff_1
lpc_coeff_2
lpc_coeff_3
lpc_resid_std
lpc_resid_var
lpc_resid_energy
dominant_freq
total_power
norm_peak_band_energy
norm_harmonics_energy
norm_peak_plus_harmonics_energy
mfcc_0
mfcc_1
mfcc_2
mfcc_3
mfcc_4
dct_mean
dct_var
hilbert_amp_mean
hilbert_freq_mean
stft_energy_mean
stft_energy_std
stft_peak_freq_mean
dwt_A4_energy
dwt_A4_std
dwt_D4_energy
dwt_D4_std
dwt_D3_energy
dwt_D3_std
dwt_D2_energy
dwt_D2_std
dwt_D1_energy
dwt_D1_std
svd_val_0
svd_val_1
svd_val_2


In [23]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
import pywt
from statsmodels.tsa.ar_model import AutoReg

class EMGFeatureExtractor:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    def extract_time_domain(self, x):
        f = {}
        # --- 1. Basic Statistics ---
        f['mean'] = np.mean(x)
        f['std'] = np.std(x)
        f['median'] = np.median(x)
        f['max'] = np.max(x)
        f['min'] = np.min(x)
        f['range'] = f['max'] - f['min']
        f['iqr'] = stats.iqr(x)
        f['kurtosis'] = stats.kurtosis(x)
        f['skewness'] = stats.skew(x)
        f['Q1'] = np.percentile(x, 25)
        f['Q3'] = np.percentile(x, 75)
        
        # --- 2. Conventional EMG Features ---
        # Integrated EMG (IEMG): Sum of absolute values
        f['integrated_EMG'] = np.sum(np.abs(x))
        
        # Mean Absolute Value (MAV)
        f['mean_absolute_value'] = np.mean(np.abs(x))
        
        # Simple Square Integral (SSI)
        f['simple_square_integral'] = np.sum(x**2)
        
        # Root Mean Square (RMS)
        f['root_mean_square'] = np.sqrt(np.mean(x**2))
        f['RMS'] = f['root_mean_square'] # Duplicate as requested
        
        # Variance
        f['variance'] = np.var(x)
        
        # Waveform Length (WL): Sum of absolute differences
        diff1 = np.diff(x)
        f['waveform_length'] = np.sum(np.abs(diff1))
        
        # Difference Absolute Mean Value (DAMV) / Mean Absolute Deviation
        f['difference_absolute_mean_value'] = np.mean(np.abs(diff1))
        
        # Difference Variance
        f['difference_variance'] = np.var(diff1)
        
        # Difference Absolute Standard Deviation (DASD)
        f['difference_absolute_standard_deviation'] = np.std(diff1)
        
        f['integrated_absolute_second_derivative'] = np.sum(np.abs(np.diff(diff1)))
        f['integrated_absolute_third_derivative'] = np.sum(np.abs(np.diff(np.diff(diff1))))
        f['integrated_exponential_absolute'] = np.sum(np.exp(np.abs(x)))
        f['integrated_absolute_log'] = np.sum(np.log(np.abs(x) + 1e-6))
        f['integrated_exponential'] = np.sum(np.exp(x))
        
        # Second Order Moment
        f['second_order_moment'] = np.mean(x**2)
        
        # Willison Amplitude (WAMP): Count differences > threshold
        # Threshold is usually dataset dependent. Using a small fixed value or heuristic.
        # For normalized data, 0.1 might be appropriate. For raw, usually 10mV+. 
        # We'll use a heuristic: 10% of std dev.
        threshold = 0.1 * f['std']
        f['Willison_amplitude'] = np.sum(np.abs(diff1) > threshold)
        
        # Myopulse Percentage Rate (MYOP): Fraction of signal > threshold
        f['myopulse_percentage_rate'] = np.sum(np.abs(x) > threshold) / len(x)

        # --- 3. Counts / Changes ---
        # Zero Crossings
        # (Counts times signal crosses mean)
        centered = x - f['mean']
        f['zero_crossings'] = np.sum(np.diff(np.signbit(centered).astype(int)) != 0)
        
        # Slope Sign Changes (SSC)
        # Change in slope direction (convex to concave or vice versa)
        # (x[i] - x[i-1]) * (x[i+1] - x[i]) < 0
        f['slope_sign_changes'] = np.sum(np.diff(np.sign(diff1)) != 0)
        
        # --- 4. Hjorth Parameters ---
        # Activity (Variance)
        f['Hjorth_activity'] = f['variance']
        
        # Mobility: sqrt(var(diff) / var(x))
        if f['variance'] > 0:
            f['Hjorth_mobility'] = np.sqrt(np.var(diff1) / f['variance'])
        else:
            f['Hjorth_mobility'] = 0
            
        # Complexity: mobility(diff) / mobility(x)
        diff2 = np.diff(diff1)
        if f['Hjorth_mobility'] > 0 and np.var(diff1) > 0:
            mob_diff = np.sqrt(np.var(diff2) / np.var(diff1))
            f['Hjorth_complexity'] = mob_diff / f['Hjorth_mobility']
        else:
            f['Hjorth_complexity'] = 0

        # --- 5. Higher Order / Integral Proxies ---
        # Integrated Exponential: sum(exp(x))
        # (Careful with overflow on raw data, best on normalized or small window)
        try:
            f['integrated_exponential'] = np.sum(np.exp(np.clip(x, -5, 5)))
        except:
            f['integrated_exponential'] = 0

        f['integrated_absolute_log'] = np.sum(np.log(np.abs(x) + 1e-6))

        return f

    def extract_frequency_domain(self, x):
        f = {}
        # FFT Coefficients (mean magnitude)
        fft_vals = np.fft.rfft(x)
        f['FFT_mean_mag'] = np.mean(np.abs(fft_vals))
        
        # AR Models (Order 4 and 7)
        for order in [4, 7]:
            try:
                res = AutoReg(x, lags=order, trend='n').fit()
                for i, c in enumerate(res.params):
                    f[f'AR_order{order}_coeff_{i}'] = c
                # PSD proxy from AR residuals
                f[f'AR_order{order}_PSD_est'] = np.var(res.resid)
            except:
                for i in range(order): f[f'AR_order{order}_coeff_{i}'] = 0
                f[f'AR_order{order}_PSD_est'] = 0
                
        return f

    def extract_decomposition_domain(self, x):
        f = {}
        # Wavelet Transform (DWT)
        # Using 'db4' level 4
        coeffs = pywt.wavedec(x, 'db4', level=4)
        coeff_names = ['A4', 'D4', 'D3', 'D2', 'D1']
        
        for name, c in zip(coeff_names, coeffs):
            f[f'wavelet_{name}_SA_sum'] = np.sum(np.abs(c))
            f[f'wavelet_{name}_SD_std'] = np.std(c)
            f[f'wavelet_{name}_VR_var'] = np.var(c)
            # Fourth Moment (unnormalized Kurtosis-like)
            f[f'wavelet_{name}_CM_moment4'] = np.mean(c**4)
            f[f'wavelet_{name}_SK_skew'] = stats.skew(c)
            f[f'wavelet_{name}_KU_kurt'] = stats.kurtosis(c)
            
        return f
    
    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))
        
        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())

print("EMGFeatureExtractor class defined.")

extractor = EMGFeatureExtractor()
dummy = np.random.randn(1000)  # 4 seconds at 250Hz

_ = extractor.list_feature_names(dummy)

EMGFeatureExtractor class defined.
Total feature count: 78
Feature names:
mean
std
median
max
min
range
iqr
kurtosis
skewness
Q1
Q3
integrated_EMG
mean_absolute_value
simple_square_integral
root_mean_square
RMS
variance
waveform_length
difference_absolute_mean_value
difference_variance
difference_absolute_standard_deviation
integrated_absolute_second_derivative
integrated_absolute_third_derivative
integrated_exponential_absolute
integrated_absolute_log
integrated_exponential
second_order_moment
Willison_amplitude
myopulse_percentage_rate
zero_crossings
slope_sign_changes
Hjorth_activity
Hjorth_mobility
Hjorth_complexity
FFT_mean_mag
AR_order4_coeff_0
AR_order4_coeff_1
AR_order4_coeff_2
AR_order4_coeff_3
AR_order4_PSD_est
AR_order7_coeff_0
AR_order7_coeff_1
AR_order7_coeff_2
AR_order7_coeff_3
AR_order7_coeff_4
AR_order7_coeff_5
AR_order7_coeff_6
AR_order7_PSD_est
wavelet_A4_SA_sum
wavelet_A4_SD_std
wavelet_A4_VR_var
wavelet_A4_CM_moment4
wavelet_A4_SK_skew
wavelet_A4_KU_kurt
wavelet_D4_

In [26]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
import scipy.fftpack as fftpack
import pywt

class MOVTriaxFeatureExtractor:
    def __init__(self, sfreq=250, vertical_axis=2):
        """
        Triaxial MOV feature extractor.

        Parameters
        ----------
        sfreq : float
            Sampling frequency (Hz).
        vertical_axis : int
            Index of the vertical acceleration axis (0, 1, or 2).
        """
        self.sfreq = sfreq
        self.vertical_axis = vertical_axis

    # --- Helper: ensure (N, 3) shape ---
    def _ensure_triax_shape(self, arr, name="acc"):
        """
        Accepts (N, 3) or (3, N) and returns (N, 3).
        """
        arr = np.asarray(arr)
        if arr.ndim != 2 or 3 not in arr.shape:
            raise ValueError(f"{name} must be 2D with one dimension = 3, got shape {arr.shape}")
        if arr.shape[1] == 3:
            return arr
        else:
            # assume (3, N)
            return arr.T

    # ---------- TIME DOMAIN ----------
    def extract_time_domain(self, acc, gyro=None):
        """
        acc : array-like, shape (N, 3) or (3, N)
        gyro : array-like, shape (N, 3) or (3, N), optional
        """
        f = {}

        acc = self._ensure_triax_shape(acc, name="acc")
        ax, ay, az = acc[:, 0], acc[:, 1], acc[:, 2]

        # --- Axis-wise stats ---
        f['triax_mean_X'] = float(np.mean(ax))
        f['triax_mean_Y'] = float(np.mean(ay))
        f['triax_mean_Z'] = float(np.mean(az))

        f['triax_variance_X'] = float(np.var(ax))
        f['triax_variance_Y'] = float(np.var(ay))
        f['triax_variance_Z'] = float(np.var(az))

        # --- Resultant magnitude ---
        mag = np.sqrt(ax**2 + ay**2 + az**2)

        # Total sum of vector magnitude over the window
        f['triax_total_sum_vector'] = float(np.sum(mag))

        # Mean magnitude (average length of the sum vector)
        f['triax_sum_vector_magnitude'] = float(np.mean(mag))

        # Activity / single magnitude area: sum of |ax|+|ay|+|az|
        f['triax_activity_single_magnitude_area'] = float(
            np.sum(np.abs(ax) + np.abs(ay) + np.abs(az))
        )

        # --- Dynamic component (subtract DC/gravity per axis) ---
        ax_d = ax - np.mean(ax)
        ay_d = ay - np.mean(ay)
        az_d = az - np.mean(az)
        mag_dyn = np.sqrt(ax_d**2 + ay_d**2 + az_d**2)
        f['triax_dynamic_sum_vector'] = float(np.sum(mag_dyn))

        # --- Vertical acceleration ---
        if self.vertical_axis not in (0, 1, 2):
            raise ValueError("vertical_axis must be 0, 1, or 2")
        a_vert = acc[:, self.vertical_axis]
        f['triax_vertical_acceleration'] = float(np.mean(np.abs(a_vert)))

        # --- Angular velocity aggregate (if gyro provided) ---
        if gyro is not None:
            gyro = self._ensure_triax_shape(gyro, name="gyro")
            gx, gy, gz = gyro[:, 0], gyro[:, 1], gyro[:, 2]
            gyro_mag = np.sqrt(gx**2 + gy**2 + gz**2)
            # Aggregate = mean magnitude over window
            f['triax_angular_velocity_aggregate'] = float(np.mean(gyro_mag))
        else:
            # If no gyro channel, set to 0 (or np.nan if you prefer)
            f['triax_angular_velocity_aggregate'] = 0.0

        return f

    # ---------- FREQUENCY DOMAIN ----------
    def extract_frequency_domain(self, acc):
        """
        Basic frequency-domain features from resultant magnitude.
        """
        f = {}

        acc = self._ensure_triax_shape(acc, name="acc")
        ax, ay, az = acc[:, 0], acc[:, 1], acc[:, 2]
        mag = np.sqrt(ax**2 + ay**2 + az**2)

        # FFT on magnitude (similar to 1D)
        fft_vals = np.fft.rfft(mag)
        freqs = np.fft.rfftfreq(len(mag), d=1/self.sfreq)

        f['triax_fft_mean_mag'] = float(np.mean(np.abs(fft_vals)))
        f['triax_fft_energy'] = float(np.sum(np.abs(fft_vals)**2))
        f['triax_dominant_freq'] = float(freqs[np.argmax(np.abs(fft_vals))])

        return f

    # ---------- DECOMPOSITION DOMAIN ----------
    def extract_decomposition_domain(self, acc):
        """
        Wavelet features on resultant magnitude (parallel to 1D extractor).
        """
        f = {}

        acc = self._ensure_triax_shape(acc, name="acc")
        ax, ay, az = acc[:, 0], acc[:, 1], acc[:, 2]
        mag = np.sqrt(ax**2 + ay**2 + az**2)

        coeffs = pywt.wavedec(mag, 'db4', level=4)
        coeff_names = ['A4', 'D4', 'D3', 'D2', 'D1']

        for name, c in zip(coeff_names, coeffs):
            f[f'triax_wavelet_{name}_energy'] = float(np.sum(c**2))
            f[f'triax_wavelet_{name}_mean'] = float(np.mean(c))
            f[f'triax_wavelet_{name}_std'] = float(np.std(c))
            f[f'triax_wavelet_{name}_skew'] = float(stats.skew(c))

        return f
    
    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))
        
        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())



class MOV1DFeatureExtractor:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    def extract_time_domain(self, x):
        f = {}
        # --- 1. Basic Statistics ---
        f['mean'] = np.mean(x)
        f['std'] = np.std(x)
        f['median'] = np.median(x)
        f['max'] = np.max(x)
        f['min'] = np.min(x)
        f['range'] = f['max'] - f['min']
        f['iqr'] = stats.iqr(x)
        f['kurtosis'] = stats.kurtosis(x)
        f['skewness'] = stats.skew(x)
        f['Q1'] = np.percentile(x, 25)
        f['Q3'] = np.percentile(x, 75)
        
        # --- 2. Temporal Features ---
        # Signal Magnitude Area (SMA) - usually sum of absolute values
        f['signal_magnitude_area'] = np.sum(np.abs(x))
        
        # Zero Crossing Rate
        centered = x - f['mean']
        f['zero_crossing_rate'] = np.sum(np.diff(np.signbit(centered).astype(int)) != 0) / len(x)
        
        # Peak Count (simple peak detection)
        # Prominence heuristic: 0.5 * std
        peaks, _ = signal.find_peaks(x, prominence=0.5*f['std'])
        f['peak_count'] = len(peaks)
        
        # --- 3. Trajectory / Angular Velocity Features ---
        # (First-Last, First-Max, etc.)
        first = x[0]
        last = x[-1]
        mx = f['max']
        mn = f['min']
        
        f['val_first_minus_last'] = first - last
        f['val_first_minus_max'] = first - mx
        f['val_first_minus_min'] = first - mn
        f['val_last_minus_max'] = last - mx
        f['val_last_minus_min'] = last - mn
        
        # --- 4. Acceleration Order (Ranking) ---
        # Rank of the last value in the sorted array (0 to 1)
        sorted_x = np.sort(x)
        f['rank_last_value'] = np.searchsorted(sorted_x, last) / len(x)
        f['rank_min_value'] = 0.0 # By definition min is rank 0
        
        return f

    def extract_frequency_domain(self, x):
        f = {}
        # FFT / DFT Coefficients
        # Mean magnitude of coefficients
        fft_vals = np.fft.rfft(x)
        f['fft_mean_mag'] = np.mean(np.abs(fft_vals))
        f['fft_energy'] = np.sum(np.abs(fft_vals)**2)
        
        # Dominant Frequency
        freqs = np.fft.rfftfreq(len(x), d=1/self.sfreq)
        f['dominant_freq'] = freqs[np.argmax(np.abs(fft_vals))]
        
        return f

    def extract_decomposition_domain(self, x):
        f = {}
        # Wavelet Transform (DWT)
        coeffs = pywt.wavedec(x, 'db4', level=4)
        coeff_names = ['A4', 'D4', 'D3', 'D2', 'D1']
        
        for name, c in zip(coeff_names, coeffs):
            f[f'wavelet_{name}_energy'] = np.sum(c**2)
            f[f'wavelet_{name}_mean'] = np.mean(c)
            f[f'wavelet_{name}_std'] = np.std(c)
            f[f'wavelet_{name}_skew'] = stats.skew(c)
            
        return f
    
    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))
        
        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())

print("MOVFeatureExtractor classes defined.")

# ---- Dummy test for MOVTriaxFeatureExtractor ----

# Create dummy triaxial accelerometer data: 1000 samples of (x,y,z)
np.random.seed(0)
acc_dummy = np.random.randn(1000, 3)  # shape (1000, 3)

# Create dummy gyro data (optional)
gyro_dummy = np.random.randn(1000, 3)

# Instantiate extractor
triax_extractor = MOVTriaxFeatureExtractor(sfreq=250, vertical_axis=2)

print("\n--- TRIAXIAL FEATURES ---")
_ = triax_extractor.list_feature_names(acc_dummy)


# ---- Dummy test for MOV1DFeatureExtractor ----

# Create dummy 1D motion signal
x_dummy = np.random.randn(1000)

# Instantiate extractor
mov1d_extractor = MOV1DFeatureExtractor(sfreq=250)

print("\n--- 1D FEATURES ---")
_ = mov1d_extractor.list_feature_names(x_dummy)


MOVFeatureExtractor classes defined.

--- TRIAXIAL FEATURES ---
Total feature count: 35
Feature names:
triax_mean_X
triax_mean_Y
triax_mean_Z
triax_variance_X
triax_variance_Y
triax_variance_Z
triax_total_sum_vector
triax_sum_vector_magnitude
triax_activity_single_magnitude_area
triax_dynamic_sum_vector
triax_vertical_acceleration
triax_angular_velocity_aggregate
triax_fft_mean_mag
triax_fft_energy
triax_dominant_freq
triax_wavelet_A4_energy
triax_wavelet_A4_mean
triax_wavelet_A4_std
triax_wavelet_A4_skew
triax_wavelet_D4_energy
triax_wavelet_D4_mean
triax_wavelet_D4_std
triax_wavelet_D4_skew
triax_wavelet_D3_energy
triax_wavelet_D3_mean
triax_wavelet_D3_std
triax_wavelet_D3_skew
triax_wavelet_D2_energy
triax_wavelet_D2_mean
triax_wavelet_D2_std
triax_wavelet_D2_skew
triax_wavelet_D1_energy
triax_wavelet_D1_mean
triax_wavelet_D1_std
triax_wavelet_D1_skew

--- 1D FEATURES ---
Total feature count: 44
Feature names:
mean
std
median
max
min
range
iqr
kurtosis
skewness
Q1
Q3
signal_magnitud

In [28]:
import time
import numpy as np
from joblib import Parallel, delayed
from pathlib import Path

# Ensure OUTPUT_DIR is defined
OUTPUT_DIR = Path("F:\Rice\Rice F25\Seizure Project\Processed_Gemini_V5_Augmented")  # Update as needed
BASE_PATH = Path("F:\Rice\Rice F25\Seizure Project")
FEATURE_DIR = BASE_PATH / 'Features'
FEATURE_DIR.mkdir(parents=True, exist_ok=True)

def extract_channel_features(extractor, signal_data):
    """
    Helper to run all extraction methods on a single 1D channel
    and return a sorted feature vector to ensure consistency.
    Works for EEG/ECG/EMG/MOV1D extractors.
    """
    feats = {}
    try:
        # Time Domain
        feats.update(extractor.extract_time_domain(signal_data))
        # Frequency Domain
        feats.update(extractor.extract_frequency_domain(signal_data))
        # Time-Frequency Domain (if available)
        if hasattr(extractor, 'extract_time_frequency_domain'):
            feats.update(extractor.extract_time_frequency_domain(signal_data))
        # Decomposition Domain
        feats.update(extractor.extract_decomposition_domain(signal_data))

        # Return values sorted by key for consistent column ordering
        return np.array([feats[k] for k in sorted(feats.keys())])
    except Exception as e:
        # Fallback: 1-dim zero marker if something goes wrong
        # (Ideally we should handle this better, but for now return zeros matching expected size roughly or just a flag)
        return np.zeros(1)


def extract_triax_mov_features(triax_extractor, acc_3axis, gyro_3axis=None):
    """
    Helper to run all extraction methods on triaxial MOV data
    (acc: (N,3), optional gyro: (N,3)) and return a sorted feature vector.
    """
    feats = {}
    try:
        # Time Domain (uses both acc + gyro if provided)
        feats.update(triax_extractor.extract_time_domain(acc_3axis, gyro=gyro_3axis))
        # Frequency Domain (on acc magnitude)
        feats.update(triax_extractor.extract_frequency_domain(acc_3axis))
        # Decomposition Domain (on acc magnitude)
        feats.update(triax_extractor.extract_decomposition_domain(acc_3axis))

        return np.array([feats[k] for k in sorted(feats.keys())])
    except Exception:
        return np.zeros(1)


def process_file_expanded_fixed(
    npz_file,
    eeg_ext,
    ecg_ext,
    emg_ext,
    mov1d_ext,
    mov_triax_ext,
    n_mov_channels=6
):
    """
    Loads an NPZ file, extracts features with FIXED MOV channel count.
    Corrected to handle X_student_bio and X_student_mov separately.
    """
    try:
        data = np.load(npz_file)
        X_t = data['X_teacher']       # Shape: (n_win, 2, 500)
        X_s_bio = data['X_student_bio'] # Shape: (n_win, 2, 500) -> ECG, EMG
        X_s_mov = data['X_student_mov'] # Shape: (n_win, 6, 50) -> 3 Acc, 3 Gyro
        y   = data['y']

        n_wins = X_t.shape[0]
        file_features = []

        for i in range(n_wins):
            win_feats = []

            # --- 1. EEG Features (Teacher) ---
            for ch_idx in range(X_t.shape[1]):
                win_feats.append(extract_channel_features(eeg_ext, X_t[i, ch_idx]))

            # --- 2. Student Bio Features ---
            # Index 0: ECG
            # Index 1: EMG
            
            # ECG
            if X_s_bio.shape[1] > 0:
                win_feats.append(extract_channel_features(ecg_ext, X_s_bio[i, 0]))
            
            # EMG
            if X_s_bio.shape[1] > 1:
                win_feats.append(extract_channel_features(emg_ext, X_s_bio[i, 1]))

            # --- 3. MOV 1D Features (per channel) ---
            # X_s_mov has shape (n_win, 6, 50)
            # Channels 0-2: Accel X,Y,Z
            # Channels 3-5: Gyro X,Y,Z

            for k in range(n_mov_channels):
                if k < X_s_mov.shape[1]:
                    feat_vec = extract_channel_features(mov1d_ext, X_s_mov[i, k])
                else:
                    # Zero padding if fewer channels than expected
                    dummy_signal = np.zeros(X_s_mov.shape[2]) 
                    feat_vec = extract_channel_features(mov1d_ext, dummy_signal)
                
                win_feats.append(feat_vec)

            # --- 4. MOV Triax Features ---
            # Accel
            acc_3axis = np.zeros((X_s_mov.shape[2], 3))
            if X_s_mov.shape[1] >= 3:
                acc_3axis = X_s_mov[i, 0:3].T # Transpose to (N, 3)
            
            # Gyro
            gyro_3axis = np.zeros((X_s_mov.shape[2], 3))
            if X_s_mov.shape[1] >= 6:
                gyro_3axis = X_s_mov[i, 3:6].T # Transpose to (N, 3)

            # Extract triax MOV features
            triax_feats = extract_triax_mov_features(mov_triax_ext, acc_3axis, gyro_3axis)
            win_feats.append(triax_feats)

            # Concatenate all features for this window
            file_features.append(np.concatenate(win_feats))

        return np.array(file_features), y

    except Exception as e:
        print(f"Error processing {npz_file.name}: {e}")
        return None, None

# --- Main Execution ---

# Re-verify Extractors (Should exist from previous cells)
if 'eeg_ex' not in locals() or 'mov1d_ex' not in locals() or 'mov_triax_ex' not in locals():
    print("Re-initializing extractors...")
    eeg_ex       = EEGFeatureExtractor(sfreq=250)
    ecg_ex       = ECGFeatureExtractor(sfreq=250)
    emg_ex       = EMGFeatureExtractor(sfreq=250)
    # Note: MOV data is 25Hz (50 samples / 2s)
    mov1d_ex     = MOV1DFeatureExtractor(sfreq=25) 
    mov_triax_ex = MOVTriaxFeatureExtractor(sfreq=25, vertical_axis=2)

if OUTPUT_DIR.exists():
    npz_files = sorted(list(OUTPUT_DIR.glob("*.npz")))
    print(f"Found {len(npz_files)} files to process with Fixed MOV logic.")
    
    print("Starting Parallel Feature Extraction (Fixed MOV=6)...")
    start_time = time.time()
    
    # Run Parallel
    results = Parallel(n_jobs=4, verbose=1)(
        delayed(process_file_expanded_fixed)(
            f, eeg_ex, ecg_ex, emg_ex, mov1d_ex, mov_triax_ex, n_mov_channels=6
        )
        for f in npz_files
    )
    
    # Consolidate
    X_expanded_list = []
    y_expanded_list = []

    for X_f, y_f in results:
        if X_f is not None and len(X_f) > 0:
            X_expanded_list.append(X_f)
            y_expanded_list.append(y_f)
            
    if X_expanded_list:
        # Check shapes before concat to be sure
        shapes = [x.shape[1] for x in X_expanded_list]
        if len(set(shapes)) > 1:
            print(f"CRITICAL WARNING: Found inconsistent feature lengths: {set(shapes)}")
            from collections import Counter
            mode_shape = Counter(shapes).most_common(1)[0][0]
            print(f"Filtering to keep only shape {mode_shape}...")
            filtered_X = []
            filtered_y = []
            for X_f, y_f in zip(X_expanded_list, y_expanded_list):
                if X_f.shape[1] == mode_shape:
                    filtered_X.append(X_f)
                    filtered_y.append(y_f)
            X_expanded_list = filtered_X
            y_expanded_list = filtered_y

        X_expanded = np.concatenate(X_expanded_list, axis=0)
        y_expanded = np.concatenate(y_expanded_list, axis=0)
        
        elapsed = time.time() - start_time
        print(f"\nExtraction Complete in {elapsed:.2f} seconds.")
        print(f"Final Feature Matrix Shape: {X_expanded.shape}")
        print(f"Final Label Vector Shape: {y_expanded.shape}")
        
        # Save (assuming FEATURE_DIR is defined elsewhere)
        out_file = FEATURE_DIR / "expanded_features_fixed.npz"
        np.savez_compressed(out_file, X_expanded=X_expanded, y_expanded=y_expanded)
        print(f"Saved expanded features to: {out_file}")
    else:
        print("No features extracted.")
else:
    print(f"Output directory not found: {OUTPUT_DIR}")

Found 2844 files to process with Fixed MOV logic.
Starting Parallel Feature Extraction (Fixed MOV=6)...


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 58.4min


KeyboardInterrupt: 

# Ibrahim's Lighter Feature Engineering

In [29]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
from scipy.integrate import simpson

class EEGFeatureExtractorLight:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    # --------- TIME DOMAIN (light) ---------
    def extract_time_domain(self, x):
        x = np.asarray(x)
        f = {}

        # Basic stats
        f['mean'] = float(np.mean(x))
        f['std'] = float(np.std(x))
        f['median'] = float(np.median(x))
        f['max'] = float(np.max(x))
        f['min'] = float(np.min(x))
        f['range'] = f['max'] - f['min']
        f['iqr'] = float(stats.iqr(x))
        f['kurtosis'] = float(stats.kurtosis(x))
        f['skew'] = float(stats.skew(x))
        f['Q1'] = float(np.percentile(x, 25))
        f['Q3'] = float(np.percentile(x, 75))

        # Temporal activity measures
        diff1 = np.diff(x)
        f['rms'] = float(np.sqrt(np.mean(x**2)))
        f['line_length'] = float(np.sum(np.abs(diff1)))

        centered = x - f['mean']
        f['zero_crossings'] = float(
            np.sum(np.diff(np.signbit(centered).astype(int)) != 0)
        )

        return f

    # --------- FREQUENCY DOMAIN (light) ---------
    def extract_frequency_domain(self, x):
        x = np.asarray(x)
        f = {}

        freqs, psd = signal.welch(x, self.sfreq, nperseg=min(len(x), 256))
        psd_sum = np.sum(psd) + 1e-12
        psd_norm = psd / psd_sum

        # Basic spectral features
        f['peak_freq'] = float(freqs[np.argmax(psd)])
        f['weighted_mean_freq'] = float(np.sum(freqs * psd_norm))
        f['median_freq'] = float(
            freqs[np.where(np.cumsum(psd_norm) >= 0.5)[0][0]]
        )

        # Band powers (delta, theta, alpha, beta)
        bands = {
            'delta': (1, 4),
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta':  (13, 30),
        }

        total_power = 0.0
        for band, (lo, hi) in bands.items():
            idx = (freqs >= lo) & (freqs <= hi)
            power = simpson(y=psd[idx], x=freqs[idx]) if np.any(idx) else 0.0
            f[f'power_{band}'] = float(power)
            total_power += power

        total_power = total_power + 1e-12

        # Relative band powers
        for band in bands.keys():
            f[f'rel_power_{band}'] = float(
                f[f'power_{band}'] / total_power
            )

        # A few simple ratios
        f['ratio_theta_alpha'] = f['power_theta'] / (f['power_alpha'] + 1e-12)
        f['ratio_alpha_beta']  = f['power_alpha'] / (f['power_beta'] + 1e-12)
        f['ratio_delta_beta']  = f['power_delta'] / (f['power_beta'] + 1e-12)

        return f

    # --------- TIME-FREQUENCY (disabled for speed) ---------
    def extract_time_frequency_domain(self, x):
        # Return empty dict to keep interface compatible
        return {}

    # --------- DECOMPOSITION (disabled for speed) ---------
    def extract_decomposition_domain(self, x):
        # Return empty dict to keep interface compatible
        return {}

    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_time_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))

        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())
    
print("EEGFeatureExtractorLight class defined.")

extractor = EEGFeatureExtractorLight()
dummy = np.random.randn(1000)  # 4 seconds at 250Hz

_ = extractor.list_feature_names(dummy)

EEGFeatureExtractorLight class defined.
Total feature count: 28
Feature names:
mean
std
median
max
min
range
iqr
kurtosis
skew
Q1
Q3
rms
line_length
zero_crossings
peak_freq
weighted_mean_freq
median_freq
power_delta
power_theta
power_alpha
power_beta
rel_power_delta
rel_power_theta
rel_power_alpha
rel_power_beta
ratio_theta_alpha
ratio_alpha_beta
ratio_delta_beta


In [30]:
import numpy as np
import scipy.signal as signal
import scipy.fftpack as fftpack
import scipy.stats as stats
from scipy.integrate import simpson

class ECGFeatureExtractorLight:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    # --- Helper: Pan-Tompkins QRS Detector (Simplified) ---
    def _detect_qrs(self, x):
        # A. Bandpass Filter (5-15Hz)
        sos = signal.butter(3, [5, 15], 'bandpass', fs=self.sfreq, output='sos')
        filtered = signal.sosfilt(sos, x)
        
        # B. Derivative
        diff = np.diff(filtered)
        
        # C. Squaring
        squared = diff ** 2
        
        # D. Moving Window Integration (150ms)
        window_size = int(0.15 * self.sfreq)
        integrated = np.convolve(squared, np.ones(window_size)/window_size, mode='same')
        
        # E. Peak Finding
        thresh = np.mean(integrated) * 2
        peaks, _ = signal.find_peaks(integrated,
                                     height=thresh,
                                     distance=int(0.2 * self.sfreq))
        return peaks

    # --- Helper: Normalized energy in peak band & harmonics ---
    def _normalized_peak_harmonic_energy(self, freqs, psd, n_harmonics=3, band_width=0.5):
        """
        Compute normalized energy around the dominant frequency and its harmonics.
        """
        freqs = np.asarray(freqs)
        psd = np.asarray(psd)

        if len(freqs) == 0:
            return 0.0, 0.0, 0.0

        total_power = simpson(y=psd, x=freqs)
        if total_power <= 0:
            return 0.0, 0.0, 0.0

        peak_idx = np.argmax(psd)
        f0 = freqs[peak_idx]

        energy_fund = 0.0
        energy_harm = 0.0

        for k in range(1, n_harmonics + 1):
            fk = k * f0
            if fk - band_width > freqs[-1]:
                break

            band_mask = (freqs >= fk - band_width) & (freqs <= fk + band_width)
            if not np.any(band_mask):
                continue

            band_energy = simpson(y=psd[band_mask], x=freqs[band_mask])

            if k == 1:
                energy_fund += band_energy
            else:
                energy_harm += band_energy

        eps = 1e-12
        norm_peak = energy_fund / (total_power + eps)
        norm_harm = energy_harm / (total_power + eps)
        norm_total = (energy_fund + energy_harm) / (total_power + eps)

        return float(norm_peak), float(norm_harm), float(norm_total)

    # ---------- TIME DOMAIN (light) ----------
    def extract_time_domain(self, x):
        x = np.asarray(x)
        f = {}

        # Basic Stats
        f['mean'] = float(np.mean(x))
        f['std'] = float(np.std(x))
        f['median'] = float(np.median(x))
        f['range'] = float(np.max(x) - np.min(x))
        f['skewness'] = float(stats.skew(x))
        f['kurtosis'] = float(stats.kurtosis(x))
        f['rms'] = float(np.sqrt(np.mean(x**2)))
        f['max'] = float(np.max(x))
        f['min'] = float(np.min(x))
        f['IQR'] = float(stats.iqr(x))
        f['Q1'] = float(np.percentile(x, 25))
        f['Q3'] = float(np.percentile(x, 75))

        # QRS & HR Features
        r_peaks = self._detect_qrs(x)

        if len(r_peaks) > 1:
            rr_intervals = np.diff(r_peaks) / self.sfreq  # seconds
            f['heart_rate'] = float(60.0 / np.mean(rr_intervals))      # BPM
            f['heart_rate_variability'] = float(np.std(rr_intervals))  # SDNN
            f['rr_range'] = float(np.max(rr_intervals) - np.min(rr_intervals))
            f['qrs_count'] = int(len(r_peaks))
        else:
            f['heart_rate'] = 0.0
            f['heart_rate_variability'] = 0.0
            f['rr_range'] = 0.0
            f['qrs_count'] = int(len(r_peaks))

        # (LPC removed for speed)

        return f

    # ---------- FREQUENCY DOMAIN (light) ----------
    def extract_frequency_domain(self, x):
        x = np.asarray(x)
        f = {}

        freqs, psd = signal.welch(x, self.sfreq, nperseg=min(len(x), 256))

        f['dominant_freq'] = float(freqs[np.argmax(psd)])
        f['total_power'] = float(np.sum(psd))

        # Normalized peak + harmonics energy
        norm_peak, norm_harm, norm_total = self._normalized_peak_harmonic_energy(
            freqs, psd, n_harmonics=3, band_width=0.5
        )
        f['norm_peak_band_energy'] = norm_peak
        f['norm_harmonics_energy'] = norm_harm
        f['norm_peak_plus_harmonics_energy'] = norm_total

        # Simple DCT summary (cheap)
        dct_val = fftpack.dct(x, type=2, norm='ortho')
        f['dct_mean'] = float(np.mean(np.abs(dct_val)))
        f['dct_var'] = float(np.var(dct_val))

        # (MFCC + Hilbert removed for speed)

        return f

    # ---------- TIME-FREQUENCY (disabled) ----------
    def extract_time_frequency_domain(self, x):
        # Empty dict to keep interface compatible
        return {}

    # ---------- DECOMPOSITION (disabled) ----------
    def extract_decomposition_domain(self, x):
        # Empty dict to keep interface compatible
        return {}

    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_time_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))

        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())

print("ECGFeatureExtractorLight class defined.")

extractor = ECGFeatureExtractorLight()
dummy = np.random.randn(1000)  # 4 seconds at 250Hz

_ = extractor.list_feature_names(dummy)


ECGFeatureExtractorLight class defined.
Total feature count: 23
Feature names:
mean
std
median
range
skewness
kurtosis
rms
max
min
IQR
Q1
Q3
heart_rate
heart_rate_variability
rr_range
qrs_count
dominant_freq
total_power
norm_peak_band_energy
norm_harmonics_energy
norm_peak_plus_harmonics_energy
dct_mean
dct_var


In [33]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal

class EMGFeatureExtractorLight:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    # ---------- TIME DOMAIN (kept, slightly cleaned) ----------
    def extract_time_domain(self, x):
        x = np.asarray(x)
        f = {}

        # --- 1. Basic Statistics ---
        f['mean'] = float(np.mean(x))
        f['std'] = float(np.std(x))
        f['median'] = float(np.median(x))
        f['max'] = float(np.max(x))
        f['min'] = float(np.min(x))
        f['range'] = f['max'] - f['min']
        f['iqr'] = float(stats.iqr(x))
        f['kurtosis'] = float(stats.kurtosis(x))
        f['skewness'] = float(stats.skew(x))
        f['Q1'] = float(np.percentile(x, 25))
        f['Q3'] = float(np.percentile(x, 75))

        # --- 2. Conventional EMG Features ---
        diff1 = np.diff(x)

        # Integrated EMG (IEMG)
        f['integrated_EMG'] = float(np.sum(np.abs(x)))

        # Mean Absolute Value (MAV)
        f['mean_absolute_value'] = float(np.mean(np.abs(x)))

        # Simple Square Integral (SSI)
        f['simple_square_integral'] = float(np.sum(x**2))

        # Root Mean Square (RMS)
        f['RMS'] = float(np.sqrt(np.mean(x**2)))

        # Variance
        f['variance'] = float(np.var(x))

        # Waveform Length (WL)
        f['waveform_length'] = float(np.sum(np.abs(diff1)))

        # Difference Absolute Mean Value (DAMV)
        f['difference_absolute_mean_value'] = float(np.mean(np.abs(diff1)))

        # Difference Variance
        f['difference_variance'] = float(np.var(diff1))

        # Difference Absolute Standard Deviation (DASD)
        f['difference_absolute_standard_deviation'] = float(np.std(diff1))

        # Higher-order difference integrals (still cheap)
        diff2 = np.diff(diff1)
        diff3 = np.diff(diff2)
        f['integrated_absolute_second_derivative'] = float(np.sum(np.abs(diff2)))
        f['integrated_absolute_third_derivative'] = float(np.sum(np.abs(diff3)))

        # Second Order Moment
        f['second_order_moment'] = float(np.mean(x**2))

        # Willison Amplitude (WAMP)
        threshold = 0.1 * f['std']
        f['Willison_amplitude'] = int(np.sum(np.abs(diff1) > threshold))

        # Myopulse Percentage Rate (MYOP)
        f['myopulse_percentage_rate'] = float(
            np.sum(np.abs(x) > threshold) / len(x)
        )

        # --- 3. Counts / Changes ---
        centered = x - f['mean']
        f['zero_crossings'] = int(
            np.sum(np.diff(np.signbit(centered).astype(int)) != 0)
        )

        f['slope_sign_changes'] = int(
            np.sum(np.diff(np.sign(diff1)) != 0)
        )

        # --- 4. Hjorth Parameters ---
        f['Hjorth_activity'] = f['variance']

        if f['variance'] > 0:
            var_diff1 = float(np.var(diff1))
            f['Hjorth_mobility'] = float(np.sqrt(var_diff1 / f['variance']))
        else:
            var_diff1 = 0.0
            f['Hjorth_mobility'] = 0.0

        if f['Hjorth_mobility'] > 0 and var_diff1 > 0:
            var_diff2 = float(np.var(diff2))
            mob_diff = np.sqrt(var_diff2 / var_diff1)
            f['Hjorth_complexity'] = float(mob_diff / f['Hjorth_mobility'])
        else:
            f['Hjorth_complexity'] = 0.0

        # --- 5. Safe exponential / log integrals (clipped) ---
        try:
            f['integrated_exponential'] = float(
                np.sum(np.exp(np.clip(x, -5, 5)))
            )
        except Exception:
            f['integrated_exponential'] = 0.0

        f['integrated_absolute_log'] = float(
            np.sum(np.log(np.abs(x) + 1e-6))
        )

        return f

    # ---------- FREQUENCY DOMAIN (very light) ----------
    def extract_frequency_domain(self, x):
        x = np.asarray(x)
        f = {}

        # Simple FFT-based features
        fft_vals = np.fft.rfft(x)
        freqs = np.fft.rfftfreq(len(x), d=1.0 / self.sfreq)
        mag = np.abs(fft_vals)

        if len(mag) > 0:
            mag_sum = np.sum(mag) + 1e-12
            f['FFT_mean_mag'] = float(np.mean(mag))
            f['FFT_total_energy'] = float(np.sum(mag**2))

            # dominant frequency
            f['FFT_dominant_freq'] = float(freqs[np.argmax(mag)])

            # spectral centroid
            f['FFT_spectral_centroid'] = float(np.sum(freqs * mag) / mag_sum)
        else:
            f['FFT_mean_mag'] = 0.0
            f['FFT_total_energy'] = 0.0
            f['FFT_dominant_freq'] = 0.0
            f['FFT_spectral_centroid'] = 0.0

        # (AR models removed for speed)
        return f

    # ---------- DECOMPOSITION (disabled) ----------
    def extract_decomposition_domain(self, x):
        # Skip wavelet features to save time
        return {}

    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))

        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())


print("EMGFeatureExtractorLight class defined.")

extractor = EMGFeatureExtractorLight(sfreq=250)
dummy = np.random.randn(1000)  # 4 seconds @ 250 Hz
_ = extractor.list_feature_names(dummy)

EMGFeatureExtractorLight class defined.
Total feature count: 36
Feature names:
mean
std
median
max
min
range
iqr
kurtosis
skewness
Q1
Q3
integrated_EMG
mean_absolute_value
simple_square_integral
RMS
variance
waveform_length
difference_absolute_mean_value
difference_variance
difference_absolute_standard_deviation
integrated_absolute_second_derivative
integrated_absolute_third_derivative
second_order_moment
Willison_amplitude
myopulse_percentage_rate
zero_crossings
slope_sign_changes
Hjorth_activity
Hjorth_mobility
Hjorth_complexity
integrated_exponential
integrated_absolute_log
FFT_mean_mag
FFT_total_energy
FFT_dominant_freq
FFT_spectral_centroid


In [32]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal

class MOVTriaxFeatureExtractorLight:
    def __init__(self, sfreq=250, vertical_axis=2):
        """
        Triaxial MOV feature extractor (lightweight).

        Parameters
        ----------
        sfreq : float
            Sampling frequency (Hz).
        vertical_axis : int
            Index of the vertical acceleration axis (0, 1, or 2).
        """
        self.sfreq = sfreq
        self.vertical_axis = vertical_axis

    # --- Helper: ensure (N, 3) shape ---
    def _ensure_triax_shape(self, arr, name="acc"):
        """
        Accepts (N, 3) or (3, N) and returns (N, 3).
        """
        arr = np.asarray(arr)
        if arr.ndim != 2 or 3 not in arr.shape:
            raise ValueError(f"{name} must be 2D with one dimension = 3, got shape {arr.shape}")
        if arr.shape[1] == 3:
            return arr
        else:
            # assume (3, N)
            return arr.T

    # ---------- TIME DOMAIN ----------
    def extract_time_domain(self, acc, gyro=None):
        """
        acc : array-like, shape (N, 3) or (3, N)
        gyro : array-like, shape (N, 3) or (3, N), optional
        """
        f = {}

        acc = self._ensure_triax_shape(acc, name="acc")
        ax, ay, az = acc[:, 0], acc[:, 1], acc[:, 2]

        # --- Axis-wise stats ---
        f['triax_mean_X'] = float(np.mean(ax))
        f['triax_mean_Y'] = float(np.mean(ay))
        f['triax_mean_Z'] = float(np.mean(az))

        f['triax_variance_X'] = float(np.var(ax))
        f['triax_variance_Y'] = float(np.var(ay))
        f['triax_variance_Z'] = float(np.var(az))

        # --- Resultant magnitude ---
        mag = np.sqrt(ax**2 + ay**2 + az**2)

        # Total sum of vector magnitude over the window
        f['triax_total_sum_vector'] = float(np.sum(mag))

        # Mean magnitude (average length of the sum vector)
        f['triax_sum_vector_magnitude'] = float(np.mean(mag))

        # Activity / single magnitude area: sum of |ax|+|ay|+|az|
        f['triax_activity_single_magnitude_area'] = float(
            np.sum(np.abs(ax) + np.abs(ay) + np.abs(az))
        )

        # --- Dynamic component (subtract DC/gravity per axis) ---
        ax_d = ax - np.mean(ax)
        ay_d = ay - np.mean(ay)
        az_d = az - np.mean(az)
        mag_dyn = np.sqrt(ax_d**2 + ay_d**2 + az_d**2)
        f['triax_dynamic_sum_vector'] = float(np.sum(mag_dyn))

        # --- Vertical acceleration ---
        if self.vertical_axis not in (0, 1, 2):
            raise ValueError("vertical_axis must be 0, 1, or 2")
        a_vert = acc[:, self.vertical_axis]
        f['triax_vertical_acceleration'] = float(np.mean(np.abs(a_vert)))

        # --- Angular velocity aggregate (if gyro provided) ---
        if gyro is not None:
            gyro = self._ensure_triax_shape(gyro, name="gyro")
            gx, gy, gz = gyro[:, 0], gyro[:, 1], gyro[:, 2]
            gyro_mag = np.sqrt(gx**2 + gy**2 + gz**2)
            f['triax_angular_velocity_aggregate'] = float(np.mean(gyro_mag))
        else:
            f['triax_angular_velocity_aggregate'] = 0.0

        return f

    # ---------- FREQUENCY DOMAIN ----------
    def extract_frequency_domain(self, acc):
        """
        Basic frequency-domain features from resultant magnitude.
        """
        f = {}

        acc = self._ensure_triax_shape(acc, name="acc")
        ax, ay, az = acc[:, 0], acc[:, 1], acc[:, 2]
        mag = np.sqrt(ax**2 + ay**2 + az**2)

        fft_vals = np.fft.rfft(mag)
        freqs = np.fft.rfftfreq(len(mag), d=1/self.sfreq)
        mag_abs = np.abs(fft_vals)

        if len(mag_abs) > 0:
            f['triax_fft_mean_mag'] = float(np.mean(mag_abs))
            f['triax_fft_energy'] = float(np.sum(mag_abs**2))
            f['triax_dominant_freq'] = float(freqs[np.argmax(mag_abs)])
        else:
            f['triax_fft_mean_mag'] = 0.0
            f['triax_fft_energy'] = 0.0
            f['triax_dominant_freq'] = 0.0

        return f

    # ---------- DECOMPOSITION DOMAIN (disabled for speed) ----------
    def extract_decomposition_domain(self, acc):
        # Skip wavelet features for the fast run
        return {}

    def list_feature_names(self, acc):
        all_features = {}
        # for triax, we need to pass gyro if you care; here we just ignore gyro
        acc = self._ensure_triax_shape(acc, name="acc")
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            # extract_time_domain expects (acc, gyro=None)
            if method is self.extract_time_domain:
                all_features.update(method(acc))
            else:
                all_features.update(method(acc))

        print("Total feature count (triax):", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())


class MOV1DFeatureExtractorLight:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    def extract_time_domain(self, x):
        x = np.asarray(x)
        f = {}

        # --- 1. Basic Statistics ---
        f['mean'] = float(np.mean(x))
        f['std'] = float(np.std(x))
        f['median'] = float(np.median(x))
        f['max'] = float(np.max(x))
        f['min'] = float(np.min(x))
        f['range'] = f['max'] - f['min']
        f['iqr'] = float(stats.iqr(x))
        f['kurtosis'] = float(stats.kurtosis(x))
        f['skewness'] = float(stats.skew(x))
        f['Q1'] = float(np.percentile(x, 25))
        f['Q3'] = float(np.percentile(x, 75))

        # --- 2. Temporal Features ---
        f['signal_magnitude_area'] = float(np.sum(np.abs(x)))

        centered = x - f['mean']
        f['zero_crossing_rate'] = float(
            np.sum(np.diff(np.signbit(centered).astype(int)) != 0) / len(x)
        )

        peaks, _ = signal.find_peaks(x, prominence=0.5 * f['std'])
        f['peak_count'] = int(len(peaks))

        # --- 3. Trajectory Features ---
        first = x[0]
        last = x[-1]
        mx = f['max']
        mn = f['min']

        f['val_first_minus_last'] = float(first - last)
        f['val_first_minus_max'] = float(first - mx)
        f['val_first_minus_min'] = float(first - mn)
        f['val_last_minus_max'] = float(last - mx)
        f['val_last_minus_min'] = float(last - mn)

        # --- 4. Acceleration Order (Ranking) ---
        sorted_x = np.sort(x)
        f['rank_last_value'] = float(np.searchsorted(sorted_x, last) / len(x))
        f['rank_min_value'] = 0.0

        return f

    def extract_frequency_domain(self, x):
        x = np.asarray(x)
        f = {}

        fft_vals = np.fft.rfft(x)
        freqs = np.fft.rfftfreq(len(x), d=1/self.sfreq)
        mag = np.abs(fft_vals)

        if len(mag) > 0:
            f['fft_mean_mag'] = float(np.mean(mag))
            f['fft_energy'] = float(np.sum(mag**2))
            f['dominant_freq'] = float(freqs[np.argmax(mag)])
        else:
            f['fft_mean_mag'] = 0.0
            f['fft_energy'] = 0.0
            f['dominant_freq'] = 0.0

        return f

    def extract_decomposition_domain(self, x):
        # Skip wavelet features for speed
        return {}

    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
            self.extract_decomposition_domain,
        ]:
            all_features.update(method(x))

        print("Total feature count (1D):", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())


print("MOVFeatureExtractorLight classes defined.")

# ---- Dummy test for MOVTriaxFeatureExtractor ----

# Create dummy triaxial accelerometer data: 1000 samples of (x,y,z)
np.random.seed(0)
acc_dummy = np.random.randn(1000, 3)  # shape (1000, 3)

# Create dummy gyro data (optional)
gyro_dummy = np.random.randn(1000, 3)

# Instantiate extractor
triax_extractor = MOVTriaxFeatureExtractorLight(sfreq=250, vertical_axis=2)

print("\n--- TRIAXIAL FEATURES ---")
_ = triax_extractor.list_feature_names(acc_dummy)


# ---- Dummy test for MOV1DFeatureExtractor ----

# Create dummy 1D motion signal
x_dummy = np.random.randn(1000)

# Instantiate extractor
mov1d_extractor = MOV1DFeatureExtractorLight(sfreq=250)

print("\n--- 1D FEATURES ---")
_ = mov1d_extractor.list_feature_names(x_dummy)


MOVFeatureExtractorLight classes defined.

--- TRIAXIAL FEATURES ---
Total feature count (triax): 15
Feature names:
triax_mean_X
triax_mean_Y
triax_mean_Z
triax_variance_X
triax_variance_Y
triax_variance_Z
triax_total_sum_vector
triax_sum_vector_magnitude
triax_activity_single_magnitude_area
triax_dynamic_sum_vector
triax_vertical_acceleration
triax_angular_velocity_aggregate
triax_fft_mean_mag
triax_fft_energy
triax_dominant_freq

--- 1D FEATURES ---
Total feature count (1D): 24
Feature names:
mean
std
median
max
min
range
iqr
kurtosis
skewness
Q1
Q3
signal_magnitude_area
zero_crossing_rate
peak_count
val_first_minus_last
val_first_minus_max
val_first_minus_min
val_last_minus_max
val_last_minus_min
rank_last_value
rank_min_value
fft_mean_mag
fft_energy
dominant_freq


In [35]:
import time
import numpy as np
from joblib import Parallel, delayed

# Ensure OUTPUT_DIR is defined
OUTPUT_DIR = Path("F:\Rice\Rice F25\Seizure Project\Processed_Gemini_V5_Augmented")  # Update as needed
BASE_PATH = Path("F:\Rice\Rice F25\Seizure Project")
FEATURE_DIR = BASE_PATH / 'Features'
FEATURE_DIR.mkdir(parents=True, exist_ok=True)

def process_file_expanded_light(
    npz_file,
    eeg_ext,
    ecg_ext,
    emg_ext,
    mov1d_ext,
    mov_triax_ext,
    n_mov_channels=6,
    bg_per_sz=2,
    max_bg_if_no_sz=500,
):
    """
    Loads an NPZ file, extracts features with FIXED MOV channel count,
    and performs LIGHT subsampling of windows:

      - Keep ALL seizure windows (y == 1)
      - Add up to `bg_per_sz` background windows per seizure
      - If no seizures in file, keep up to `max_bg_if_no_sz` background windows

    Parameters
    ----------
    npz_file : Path-like
        .npz file containing X_teacher, X_student_bio, X_student_mov, y
    eeg_ext, ecg_ext, emg_ext, mov1d_ext, mov_triax_ext :
        Feature extractor instances with extract_* methods.
    n_mov_channels : int
        Expected number of MOV channels (1D) per window (usually 6).
    bg_per_sz : int
        Number of background windows to sample per seizure window.
    max_bg_if_no_sz : int
        Max number of background windows to keep if no seizures present.

    Returns
    -------
    file_features : np.ndarray of shape (n_kept_windows, n_features)
    y_kept        : np.ndarray of shape (n_kept_windows,)
    """
    try:
        data = np.load(npz_file)
        X_t     = data['X_teacher']       # (n_win, n_eeg_ch, n_samples_eeg)
        X_s_bio = data['X_student_bio']   # (n_win, 2, n_samples_bio) -> ECG, EMG
        X_s_mov = data['X_student_mov']   # (n_win, 6, n_samples_mov) -> 3 Acc, 3 Gyro
        y       = data['y']               # (n_win,)

        n_wins = X_t.shape[0]

        # ---------- 1. Choose which windows to keep ----------
        idx_all = np.arange(n_wins)
        idx_sz  = idx_all[y == 1]
        idx_bg  = idx_all[y == 0]

        keep_idx = []

        if len(idx_sz) > 0:
            # Keep all seizure windows
            keep_idx.extend(idx_sz.tolist())

            # Sample background windows relative to number of seizures
            n_bg_target = min(len(idx_bg), bg_per_sz * len(idx_sz))
            if n_bg_target > 0:
                bg_sample = np.random.choice(idx_bg, size=n_bg_target, replace=False)
                keep_idx.extend(bg_sample.tolist())
        else:
            # No seizures in this file: keep a capped number of background windows
            n_bg_target = min(len(idx_bg), max_bg_if_no_sz)
            if n_bg_target > 0:
                bg_sample = np.random.choice(idx_bg, size=n_bg_target, replace=False)
                keep_idx.extend(bg_sample.tolist())

        # If nothing to keep, skip this file
        if not keep_idx:
            return None, None

        keep_idx = np.array(keep_idx, dtype=int)
        # Shuffle so seizures and backgrounds are mixed
        np.random.shuffle(keep_idx)

        # ---------- 2. Feature extraction on selected windows ----------
        file_features = []

        for i in keep_idx:
            win_feats = []

            # --- 1. EEG Features (Teacher) ---
            # X_t[i] shape: (n_eeg_ch, n_samples_eeg)
            for ch_idx in range(X_t.shape[1]):
                win_feats.append(extract_channel_features(eeg_ext, X_t[i, ch_idx]))

            # --- 2. Student Bio Features ---
            # Index 0: ECG
            # Index 1: EMG
            if X_s_bio.shape[1] > 0:
                # ECG
                win_feats.append(extract_channel_features(ecg_ext, X_s_bio[i, 0]))

            if X_s_bio.shape[1] > 1:
                # EMG
                win_feats.append(extract_channel_features(emg_ext, X_s_bio[i, 1]))

            # --- 3. MOV 1D Features (per channel) ---
            # X_s_mov shape: (n_win, 6, n_samples_mov)
            # Channels 0-2: Accel X,Y,Z
            # Channels 3-5: Gyro X,Y,Z
            for k in range(n_mov_channels):
                if k < X_s_mov.shape[1]:
                    sig = X_s_mov[i, k]
                else:
                    # Zero padding if fewer channels than expected
                    sig = np.zeros(X_s_mov.shape[2], dtype=float)

                feat_vec = extract_channel_features(mov1d_ext, sig)
                win_feats.append(feat_vec)

            # --- 4. MOV Triax Features ---
            # Accel (first 3 channels)
            n_mov_samples = X_s_mov.shape[2]
            acc_3axis = np.zeros((n_mov_samples, 3), dtype=float)
            if X_s_mov.shape[1] >= 3:
                acc_3axis = X_s_mov[i, 0:3].T  # (N, 3)

            # Gyro (next 3 channels)
            gyro_3axis = np.zeros((n_mov_samples, 3), dtype=float)
            if X_s_mov.shape[1] >= 6:
                gyro_3axis = X_s_mov[i, 3:6].T  # (N, 3)

            triax_feats = extract_triax_mov_features(mov_triax_ext, acc_3axis, gyro_3axis)
            win_feats.append(triax_feats)

            # Concatenate all feature vectors for this window
            file_features.append(np.concatenate(win_feats))

        file_features = np.array(file_features)
        y_kept = y[keep_idx]

        return file_features, y_kept

    except Exception as e:
        print(f"Error processing {getattr(npz_file, 'name', npz_file)}: {e}")
        return None, None



# Ensure the list of files is available
if 'npz_files' not in locals():
    npz_files = sorted(list(OUTPUT_DIR.glob("*.npz")))

# Select a subset of 3 files
subset_files = npz_files[:3]
print(f"Running benchmark on {len(subset_files)} files...")

start_time_subset = time.time()

# Run extraction on the subset
results_subset = Parallel(n_jobs=4, verbose=1)(
    delayed(process_file_expanded_light)(
        f, eeg_ex, ecg_ex, emg_ex, mov1d_ex, mov_triax_ex, n_mov_channels=6
    )
    for f in subset_files
)

end_time_subset = time.time()
elapsed_subset = end_time_subset - start_time_subset

# Calculate total windows processed
total_windows_subset = 0
valid_files_count = 0

for X_f, y_f in results_subset:
    if X_f is not None:
        total_windows_subset += X_f.shape[0]
        valid_files_count += 1

if total_windows_subset > 0:
    time_per_window = elapsed_subset / total_windows_subset
    
    target_total_windows = 562817
    estimated_total_seconds = time_per_window * target_total_windows
    estimated_hours = estimated_total_seconds / 3600
    
    print(f"\n--- Benchmark Results ---")
    print(f"Files Processed: {valid_files_count}")
    print(f"Total Windows in Subset: {total_windows_subset}")
    print(f"Time Taken: {elapsed_subset:.2f} seconds")
    print(f"Avg Time per Window: {time_per_window:.4f} seconds")
    print(f"\n--- Projection ---")
    print(f"Estimated Time for {target_total_windows} windows: {estimated_hours:.2f} hours")
else:
    print("Benchmark failed: No windows extracted from the subset.")

Running benchmark on 3 files...


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.



--- Benchmark Results ---
Files Processed: 3
Total Windows in Subset: 1144
Time Taken: 26.18 seconds
Avg Time per Window: 0.0229 seconds

--- Projection ---
Estimated Time for 562817 windows: 3.58 hours


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   26.1s finished


In [39]:
import numpy as np
import pickle
from pathlib import Path
from joblib import Parallel, delayed
import time

# Define paths
OUTPUT_DIR = Path("F:\Rice\Rice F25\Seizure Project\Processed_Gemini_V5_Augmented")  # Update as needed
BASE_PATH = Path("F:\Rice\Rice F25\Seizure Project")
DATA_DIR = BASE_PATH / "Processed_Gemini_V5_Augmented"
FEATURE_DIR = BASE_PATH / "Features"
FEATURE_DIR.mkdir(parents=True, exist_ok=True)
MANIFEST_FILE = FEATURE_DIR / "processing_manifest.pkl"
OUTPUT_FILE = FEATURE_DIR / "lightweight_features.npz"

# Load Manifest
try:
    with open(MANIFEST_FILE, 'rb') as f:
        manifest = pickle.load(f)
    print(f"Loaded manifest with {len(manifest)} files to process.")
except FileNotFoundError:
    print(f"Manifest file {MANIFEST_FILE} not found.")
    manifest = {}

# --- Helper Functions ---

def extract_channel_features(extractor, signal_data):
    """Runs all extraction methods on a single 1D channel."""
    feats = {}
    try:
        feats.update(extractor.extract_time_domain(signal_data))
        feats.update(extractor.extract_frequency_domain(signal_data))
        if hasattr(extractor, 'extract_time_frequency_domain'):
            feats.update(extractor.extract_time_frequency_domain(signal_data))
        if hasattr(extractor, 'extract_decomposition_domain'):
            feats.update(extractor.extract_decomposition_domain(signal_data))
        # Return sorted values for consistency
        return np.array([feats[k] for k in sorted(feats.keys())])
    except Exception:
        return None

def extract_triax_mov_features(extractor, acc, gyro=None):
    """Runs extraction on triaxial MOV data."""
    feats = {}
    try:
        feats.update(extractor.extract_time_domain(acc, gyro=gyro))
        feats.update(extractor.extract_frequency_domain(acc))
        feats.update(extractor.extract_decomposition_domain(acc))
        return np.array([feats[k] for k in sorted(feats.keys())])
    except Exception:
        return None

def process_file_manifest(filename, indices, extractors, data_dir, n_mov_channels=6):
    """Worker function to process specific indices of a file."""
    eeg_ex, ecg_ex, emg_ex, mov1d_ex, mov_triax_ex = extractors
    file_path = data_dir / filename
    
    try:
        with np.load(file_path) as data:
            # Load and subset data
            X_t     = data['X_teacher'][indices]        # (n, 2, 500)
            X_s_bio = data['X_student_bio'][indices]    # (n, 2, 500)
            X_s_mov = data['X_student_mov'][indices]    # (n, 6, 50)
            y       = data['y'][indices]                # (n,)

        n_wins = len(y)
        file_feats  = []
        file_labels = []     # <--- new

        for i in range(n_wins):
            win_feats = []
            valid_window = True

            # 1. EEG (Teacher) - 2 channels
            for ch in range(X_t.shape[1]):
                f = extract_channel_features(eeg_ex, X_t[i, ch])
                if f is None:
                    valid_window = False
                    break
                win_feats.append(f)
            if not valid_window:
                continue

            # 2. ECG (Student Bio ch 0)
            f_ecg = extract_channel_features(ecg_ex, X_s_bio[i, 0])
            if f_ecg is None:
                continue
            win_feats.append(f_ecg)

            # 3. EMG (Student Bio ch 1)
            f_emg = extract_channel_features(emg_ex, X_s_bio[i, 1])
            if f_emg is None:
                continue
            win_feats.append(f_emg)

            # 4. MOV 1D (Student Mov ch 0-5)
            for k in range(n_mov_channels):
                if k < X_s_mov.shape[1]:
                    sig = X_s_mov[i, k]
                else:
                    sig = np.zeros(X_s_mov.shape[2], dtype=float)

                f_mov = extract_channel_features(mov1d_ex, sig)
                if f_mov is None:
                    valid_window = False
                    break
                win_feats.append(f_mov)

            if not valid_window:
                continue

            # 5. MOV Triax
            acc_3 = X_s_mov[i, 0:3].T if X_s_mov.shape[1] >= 3 else np.zeros((X_s_mov.shape[2], 3))
            gyro_3 = X_s_mov[i, 3:6].T if X_s_mov.shape[1] >= 6 else np.zeros((X_s_mov.shape[2], 3))

            f_triax = extract_triax_mov_features(mov_triax_ex, acc_3, gyro_3)
            if f_triax is None:
                continue
            win_feats.append(f_triax)

            # If we get here, the window is valid: add features + its label
            file_feats.append(np.concatenate(win_feats))
            file_labels.append(y[i])

        if len(file_feats) == 0:
            return None, None

        return np.array(file_feats), np.array(file_labels)

    except Exception as e:
        # print(f"Error processing {filename}: {e}")
        return None, None


# --- Execution ---

# Instantiate Extractors (assuming classes are defined in kernel)
print("Initializing extractors...")
eeg_ex = EEGFeatureExtractorLight(sfreq=250)
ecg_ex = ECGFeatureExtractorLight(sfreq=250)
emg_ex = EMGFeatureExtractorLight(sfreq=250)
mov1d_ex = MOV1DFeatureExtractorLight(sfreq=25)
mov_triax_ex = MOVTriaxFeatureExtractorLight(sfreq=25, vertical_axis=2)

extractors_tuple = (eeg_ex, ecg_ex, emg_ex, mov1d_ex, mov_triax_ex)

if manifest:
    print(f"Starting parallel extraction on {len(manifest)} files...")
    start_time = time.time()
    
    # Run Parallel
    # Using n_jobs=4 to balance CPU/Memory
    results = Parallel(n_jobs=4, verbose=5)( 
        delayed(process_file_manifest)(fname, idxs, extractors_tuple, DATA_DIR)
        for fname, idxs in manifest.items()
    )
    
    elapsed = time.time() - start_time
    print(f"Extraction finished in {elapsed:.2f} seconds.")

    # Aggregate Results
    X_list = []
    y_list = []
    
    print("Aggregating results...")
    for X_f, y_f in results:
        if X_f is not None and len(X_f) > 0:
            X_list.append(X_f)
            y_list.append(y_f)
            
    if X_list:
        # Check consistency
        shapes = [x.shape[1] for x in X_list]
        if len(set(shapes)) > 1:
            print(f"Warning: Inconsistent feature shapes detected: {set(shapes)}")
            mode_shape = max(set(shapes), key=shapes.count)
            print(f"Filtering for mode shape: {mode_shape}")
            
            X_list_clean = []
            y_list_clean = []
            for x, y in zip(X_list, y_list):
                if x.shape[1] == mode_shape:
                    X_list_clean.append(x)
                    y_list_clean.append(y)
            X_list = X_list_clean
            y_list = y_list_clean

        X_final = np.concatenate(X_list, axis=0)
        y_final = np.concatenate(y_list, axis=0)
        
        print(f"Final Feature Matrix Shape: {X_final.shape}")
        print(f"Final Label Vector Shape: {y_final.shape}")
        
        # Save
        np.savez_compressed(OUTPUT_FILE, X=X_final, y=y_final)
        print(f"Saved optimized features to {OUTPUT_FILE}")
    else:
        print("No features extracted successfully.")
else:
    print("Manifest is empty or not loaded.")

Loaded manifest with 2699 files to process.
Initializing extractors...
Starting parallel extraction on 2699 files...


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    7.1s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  4.2min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  8.3min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 14.3min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed: 17.1min
[Parallel(n_jobs=4)]: Done 874 tasks      | elapsed: 18.7min
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed: 20.0min
[Parallel(n_jobs=4)]: Done 1544 tasks      | elapsed: 20.8min
[Parallel(n_jobs=4)]: Done 1928 tasks      | elapsed: 21.5min
[Parallel(n_jobs=4)]: Done 2306 tasks      | elapsed: 25.9min
[Parallel(n_jobs=4)]: Done 2692 out of 2699 | elapsed: 26.8min remaining:    4.1s
[Parallel(n_jobs=4)]: Done 2699 out of 2699 | elapsed: 26.9min finished


Extraction finished in 1612.71 seconds.
Aggregating results...
Final Feature Matrix Shape: (562729, 274)
Final Label Vector Shape: (562729,)
Saved optimized features to F:\Rice\Rice F25\Seizure Project\Features\lightweight_features.npz


# SeizeIT2 Replicated Feature Engineering - SVM

In [ ]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
from scipy.integrate import simpson
import antropy as ant
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
import pywt

class EEGFeatureExtractorSeizeIT2:
    def __init__(self, sfreq=250):
        self.sfreq = sfreq

    # --- Helper: Compute Power Band Asymmetry ---


    def extract_time_domain(self, x):
        f = {}
        # 1. Statistics
        f['max'] = np.max(x)
        f['min'] = np.min(x)
        f['kurtosis'] = stats.kurtosis(x)
        f['skew'] = stats.skew(x)
            
        # 4. Nonlinear / Entropy 
        f['sample_entropy'] = ant.sample_entropy(x)
        f['spectral_entropy'] = ant.spectral_entropy(x, self.sfreq, method='welch', normalize=True)
        f['shannon_entropy'] = stats.entropy(np.abs(x) + 1e-6)
        
        # 6. Energy/Power
        f['rms'] = np.sqrt(np.mean(x**2))
        
        # 7. Patterns
        f['zero_crossings'] = ant.num_zerocross(x)
        
        return f

    def extract_frequency_domain(self, x):
        f = {}
        freqs, psd = signal.welch(x, self.sfreq, nperseg=min(len(x), 256))
        psd_norm = psd / (np.sum(psd) + 1e-6)
        
        bands = {'delta': (1, 3), 'theta': (4, 8), 'alpha': (9, 13), 'beta': (14, 20)}
        f['mean_power_delta'] = np.mean(psd[(freqs >= bands['delta'][0]) & (freqs < bands['delta'][1])])
        f['mean_power_theta'] = np.mean(psd[(freqs >= bands['theta'][0]) & (freqs < bands['theta'][1])])
        f['mean_power_alpha'] = np.mean(psd[(freqs >= bands['alpha'][0]) & (freqs < bands['alpha'][1])])
        f['mean_power_beta'] = np.mean(psd[(freqs >= bands['beta'][0]) & (freqs < bands['beta'][1])])
        
        f['normalized_power_delta'] = np.sum(psd_norm[(freqs >= bands['delta'][0]) & (freqs < bands['delta'][1])])
        f['normalized_power_theta'] = np.sum(psd_norm[(freqs >= bands['theta'][0]) & (freqs < bands['theta'][1])])
        f['normalized_power_alpha'] = np.sum(psd_norm[(freqs >= bands['alpha'][0]) & (freqs < bands['alpha'][1])])
        f['normalized_power_beta'] = np.sum(psd_norm[(freqs >= bands['beta'][0]) & (freqs < bands['beta'][1])])
        
        f['mean_power_hf_band'] = np.mean(psd[(freqs >= 40) and (freqs <= 80)])
        f['normalized_power_hf_band'] = np.sum(psd_norm[(freqs >= 40) and (freqs <= 80)])
        
        return f
    
    def list_feature_names(self, x):
        all_features = {}
        for method in [
            self.extract_time_domain,
            self.extract_frequency_domain,
        ]:
            all_features.update(method(x))
        
        print("Total feature count:", len(all_features))
        print("Feature names:")
        for name in all_features:
            print(name)

        return list(all_features.keys())


print("EEGFeatureExtractorSeizeIT2 updated with Time, Frequency, Time-Freq, and Decomposition domains.")

extractor = EEGFeatureExtractorSeizeIT2()
dummy = np.random.randn(1000)  # 4 seconds at 250Hz

_ = extractor.list_feature_names(dummy)